In [44]:
import pandas as pd
from PIL import Image
import numpy as np
import torch
import torch.nn as nn  
import numpy as np 
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os

In [ ]:
class ArabicDataset(Dataset): 
    def __init__(self, csv_file, root_dir, transform=None): 
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir 
        self.transform = transform 
    def __len__(self): 
        return len(self.data_frame) 
    def __getitem__(self, idx): 
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        # Check if the file exists
        if not os.path.exists(img_name):
            raise FileNotFoundError(f"File {img_name} not found")
        
        image = Image.open(img_name).convert('L')  # Convert to grayscale if needed
        
        label = self.data_frame.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label
# Data transformations 

# Data augmentation 
transform = transforms.Compose([ 
    transforms.ToTensor(), 
    
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.Normalize((0.5,), (0.5,)) ])


dataset = ArabicDataset(csv_file='data/data.csv', root_dir='data/images', transform=transform)

# Calculate the lengths for each split
train_size = int(0.6 * len(dataset))
test_size = int(0.2 * len(dataset))
validation_size = len(dataset) - train_size - test_size

# Split the dataset
train_dataset, test_dataset, validation_dataset = random_split(dataset, [train_size, test_size, validation_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [ ]:
class ArabicNet(nn.Module):
    def __init__(self, initial_size=None):
        super(ArabicNet, self).__init__()
        self.encoder =  nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), 

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2), 

            nn.Conv2d(128, 256, kernel_size=3), 
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.decoder = nn.Sequential( 
            nn.Flatten(), 
            nn.Linear(256 * 2 * 2 , 256),
            nn.ReLU(), 
            nn.Dropout(0.5),
            nn.Linear(256, 128), 
            nn.ReLU(), 
            nn.Dropout(0.5),
            nn.Linear(128, 64), 
            nn.ReLU(), 
            nn.Linear(64, 32)
        ) 
        
    def forward(self, x):
        x = self.encoder(x) 
        # print(x.shape)
        x = self.decoder(x) 
        return x
   



In [ ]:
# Example of how to create the model 
model = ArabicNet() 
# Define a loss function and optimizer 
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [56]:
num_epochs = 15
best_val_loss = float('inf') 
patience = 3 
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss:.4f}, Accuracy: {100 * correct / total:.2f}%')
    
    #Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter+=1
        if patience_counter >= patience:
            print(f"Early stop with {patience_counter} counter patience and {patience} ppatience")
            break

print("Training is done :)")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x4096 and 256x256)

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # print(f"The model predicts {predicted}, and the actual label is {labels}")
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

In [54]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out
    
class ArabicResNet(nn.Module):
    def __init__(self, initial_size=None):
        super(ArabicResNet, self).__init__()
        self.in_channels = 32
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(ResidualBlock, 32, 2, stride=1)
        self.layer2 = self._make_layer(ResidualBlock, 64, 2, stride=2)
        self.layer3 = self._make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer4 = self._make_layer(ResidualBlock, 256, 2, stride=2)
        
        self.decoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 , 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
        )

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.decoder(x)
        return x

In [55]:
# Example of how to create the model 
model = ArabicResNet() 
# Define a loss function and optimizer 
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)